In [1]:
import os
os.chdir("..")
import pickle
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt  
from matplotlib import colors

import imgkit


In [6]:
s1 = pd.read_pickle("OUTPUT/opt_settings1.pkl").loc[:,['transaction_cost', 'lag', 'resample',
       'rounding', 'windowZ', 'windowOLS',  'entryZ', 'exitZ','entryMult', 'exitMult']]
s2 = pd.read_pickle("OUTPUT/opt_settings.pkl").loc[:,['transaction_cost', 'lag', 'resample',
       'rounding', 'windowZ', 'windowOLS',  'entryZ', 'exitZ']]

s2['entryMult']=0
s2['exitMult'] =0
s = s2.append(s1[~np.in1d(s1.index.get_level_values('Model'),"Benchmark")])
r = pd.read_pickle("OUTPUT/results2.pkl").loc[:,['sharpe_ratio', 'total_return', 
                        'max_drawdown', 'mean_hold_time', 'n_transactions',
       'long_ret', 'short_ret', 'spread_ret', 'trans_ret']]

ts = ['T1','T3','T5']
#ts = ['T0','T1','T2','T3','T4','T5','T6']
s = s.droplevel("Sample")[np.in1d(s.index.get_level_values('Treatment'),ts)]
r = r[np.in1d(r.index.get_level_values('Treatment'),ts)]
r.reorder_levels(['Data','Model','Sample','Treatment'])
r = r.sort_index(level = ['Sample'])
s = s.sort_index(level = ['Data','Model'])
skal = np.in1d(s.index.get_level_values('Model'),"Kalman")
rkal = np.in1d(r.index.get_level_values('Model'),"Kalman")
srya = np.in1d(s.index.get_level_values('Data'),"rya_ryaay")
rrya = np.in1d(r.index.get_level_values('Data'),"rya_ryaay")
shsbc= np.in1d(s.index.get_level_values('Data'),"hsbcln_hsbc")
rhsbc= np.in1d(r.index.get_level_values('Data'),"hsbcln_hsbc")
r[r[['long_ret','short_ret','spread_ret','trans_ret']] ==0] = 1
s2 = s.copy()
r2 = r.copy()

r.sharpe_ratio = ["0.00" if math.isnan(val) else "{0:.2f}".format(val)  for val in r.sharpe_ratio]
r.total_return = ["0.00%" if math.isnan(val) else "{0:.2f}%".format(val*100)  for val in r.total_return]
r.max_drawdown = ["0.00%" if math.isnan(val) else "{0:.2f}%".format(val*100)  for val in r.max_drawdown]
r.mean_hold_time = r.mean_hold_time.astype(int)
r.n_transactions = r.n_transactions.astype(int)
r.long_ret   =["0.00%" if math.isnan(val) else "{0:.2f}%".format((val-1)*100)  for val in r.long_ret]
r.short_ret  =["0.00%" if math.isnan(val) else "{0:.2f}%".format((val-1)*100)  for val in r.short_ret]
r.spread_ret =["0.00%" if math.isnan(val) else "{0:.2f}%".format((val-1)*100)  for val in r.spread_ret]
r.trans_ret  =["0.00%" if math.isnan(val) else "{0:.2f}%".format((val-1)*100)  for val in r.trans_ret]
s.transaction_cost = ["{0:.1f} bps".format(val*10000)  for val in s.transaction_cost]
s.lag = s.lag.astype(int)
s['resample'] = s['resample'].astype(int)
s.rounding = s.rounding.astype(int)
s.exitMult   = ["0" if math.isnan(val) else "{0:.1f}".format(val)  for val in s.exitMult]
s.entryMult  = ["0" if math.isnan(val) else "{0:.1f}".format(val)  for val in s.entryMult]
s.exitZ      = ["0" if math.isnan(val) else "{0:.1f}".format(val)  for val in s.exitZ]
s.entryZ     = ["0" if math.isnan(val) else "{0:.1f}".format(val)  for val in s.entryZ]
s.windowZ    = ["0" if math.isnan(val) else int(val)  for val in s.windowZ]
s.windowOLS  = ["0" if math.isnan(val) else int(val)  for val in s.windowOLS]

In [3]:
def b_g(x,  filt=None,r = True,cmap='RdYlBu', low=0, high=0):
    if filt is None:
        filt = [True] * len(x)
    # Pass the columns from Dataframe A 
    if r:
        a = r2.loc[filt,x.name].copy()
    else:
        a = s2.loc[filt,x.name].copy()
    rng = a.max() - a.min()
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]


In [4]:
c1 = ['sharpe_ratio', 'max_drawdown', 'mean_hold_time','n_transactions']
c2 = ['total_return','long_ret', 'short_ret', 'spread_ret', 'trans_ret']
stylestr = "* {font-family: \"Times New Roman\", Times, serif;} table, td, tr, th {text-align: center;border-color: black;border-width:thin;border-style:solid; border-width: 2px;border-collapse:collapse}"
widthstr = "table {table-layout: fixed; width: 200px;} th,td {width: 120px;overflow: hidden;}"


In [5]:
for t in ["test","train"]:
    for m in ["Benchmark","Kalman"]:
        for c in [1,2]:
            cols = c1 if c ==1 else c2
            f = np.in1d(r.index.get_level_values('Model'),m) & np.in1d(r.index.get_level_values('Sample'),t)
            html = r.droplevel("Model").droplevel("Sample").loc[f,cols].style.apply(b_g, filt = f).render()
            html = html[0:27] + stylestr + widthstr + html[27:len(html)]
            path = "OUTPUT/Tables/r_" + m + "_" + t + "_" + str(c) + ".png"
            imgkit.from_string(html, path)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


/Users/Ferni/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


In [96]:
f = np.in1d(s.index.get_level_values('Model'),"Kalman")
html = s.loc[f,['transaction_cost', 'lag', 'resample', 'rounding', 'entryMult', 'exitMult']]\
            .droplevel("Model").style.apply(b_g,  r=False, filt = f).render()
html = html[0:27] + stylestr + html[27:len(html)]
path = "OUTPUT/Tables/s_all_Kalman.png"
imgkit.from_string(html, path)

html = s.loc[~f,['transaction_cost', 'lag', 'resample', 'rounding', 'windowZ','windowOLS', 'entryZ', 'exitZ']]\
            .droplevel("Model").style.apply(b_g,  r=False, filt = ~f).render()
html = html[0:27] + stylestr + html[27:len(html)]
path = "OUTPUT/Tables/s_all_Benchmark.png"
imgkit.from_string(html, path)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [82]:
for f1 in [True,False]:
    for f2 in [True,False]:
        filt1 = skal if f1 else ~skal
        filt2 = srya if f2 else shsbc
        filt = filt1 & filt2
        
        cols = (['transaction_cost', 'lag', 'resample', 'rounding', 'entryMult', 'exitMult'] if f1 else 
                ['transaction_cost', 'lag', 'resample', 'rounding', 'windowZ','windowOLS', 'entryZ', 'exitZ'])
        
        html = s.loc[filt,cols].droplevel("Data").droplevel("Model").dropna('columns').style.apply(b_g, filt=filt, r=False).render()
        html = html[0:27] + stylestr + html[27:len(html)]
        
        d1 = "Kalman" if f1 else "Benchmark"
        d2 = "Ryanair" if f2 else "HSBC"
        path = "OUTPUT/Tables/s_"+d1+"_"+d2+".png"
        imgkit.from_string(html, path)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


In [7]:
for f1 in [True,False]:
    for f2 in [True,False]:
        for c in [1,2]:
            cols = c1 if c ==1 else c2
            filt1 = rkal if f1 else ~rkal
            filt2 = rrya if f2 else rhsbc
            filt = filt1 & filt2

            html = r.droplevel("Data").droplevel("Model").loc[filt,cols].style.apply(b_g, filt=filt).render()
            html = html[0:27] + stylestr+widthstr + html[27:len(html)]

            d1 = "Kalman" if f1 else "Benchmark"
            d2 = "Ryanair" if f2 else "HSBC"
            path = "OUTPUT/Tables/r_"+d1+"_"+d2+"_"+str(c)+".png"
            imgkit.from_string(html, path)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


/Users/Ferni/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


/Users/Ferni/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
